After optimizing the parameters for visibility approximation, we need to test how good such approximation is.

In [ ]:
%reset -f
%matplotlib notebook
%reload_ext autoreload
%autoreload 2
import os
import numpy as np
from tqdm import tqdm_notebook

import add_path
import sampler
import visibility as vis
import fov_approx_gp as fgp

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
from IPython import display
plotly.offline.init_notebook_mode(connected=True)

n_test_fs = 300
n_test_pws = 20

# test the GP approximator just generated
uname = 'fov45_fs70_lm1000_k15'
param_dir = '/home/zichao/sources/FIF_ws/src/rpg_information_field/act_map/trace'
abs_data_dir = os.path.join(param_dir, fgp.FoVApproximatorGP.base_savedir_nm, uname)
assert os.path.exists(abs_data_dir), "{0} does not exist.".format(abs_data_dir)
print('Will load fov approximator from {0}.'.format(abs_data_dir))

# alternatively you can also test the provided approximator under params
# uname = 'fov45_fs150_lm1000_k15'
# param_dir = '/home/zichao/sources/FIF_ws/src/rpg_information_field/act_map/params'
# abs_data_dir = os.path.join(param_dir, fgp.FoVApproximatorGP.base_savedir_nm, uname)
# assert os.path.exists(abs_data_dir), "{0} does not exist.".format(abs_data_dir)
# print('Will load fov approximator from {0}.'.format(abs_data_dir))

fov_approx = fgp.FoVApproximatorGP(abs_data_dir)

In [ ]:
test_pws = sampler.randomPointsOnSphere(n_test_pws, 3)
test_fs = sampler.randomPointsOnSphere(n_test_fs, 1)

gt_soft_vis = []
pred_soft_vis = []
fs_angle = []
for pw in tqdm_notebook(test_pws, desc='points'):
    Y = fov_approx.calculateY(pw)
    c = np.dot(fov_approx.invK, Y)
    fs_angle_i = [vis.angleDegBetween(pw, v) for v in test_fs]
    fs_angle.extend(fs_angle_i)
    
    gt_soft_vis_i = [vis.softVisibility(v, pw, fov_approx.hfov_rad, fov_approx.sigmoid_k, fov_approx.fast_sigmoid)
                     for v in test_fs]
    gt_soft_vis.extend(gt_soft_vis_i)
    
    pred_soft_vis_i = []
    for f in test_fs:
        Kreg = fov_approx.calculateKreg(f)
        pred_soft_vis_i.append(np.asscalar(np.dot(Kreg, c)))
    pred_soft_vis.extend(pred_soft_vis_i)
    
err = np.array(gt_soft_vis) - np.array(pred_soft_vis)
print("The error: {0} - {1}".format(np.min(err), np.max(err)))

trace_gt = go.Scatter(x=fs_angle, y=gt_soft_vis, mode = 'markers', name='Soft Visibility Groundtruth')
trace_predict = go.Scatter(x=fs_angle, y=pred_soft_vis, mode = 'markers', name='Predicted',
                           marker=dict(color='rgba(0, 120, 0, 0.3)'))

fig = plotly.tools.make_subplots(rows=1, cols=1)
fig.append_trace(trace_gt, 1, 1)
fig.append_trace(trace_predict, 1, 1)
iplot(fig)